In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from SSLRS.dataset import *
import pandas as pd
from fastai.distributed import *
from fastai.vision.all import DataLoader
import torch
import SSLRS.utils as utils
from SSLRS.visiontrans import DINOHead
from torch.utils.data import Dataset
# dl1 = DistributedDL(dls, 0, 4)

In [ ]:
df=pd.read_csv('./data/file.csv')
newdf=df[df['Isval']==0].sample(frac=0.05,random_state=10)
newdf['labels']=1

In [ ]:
newdf.head()

,fname,Isval,label,labels
178815,./data/BigEarthNet-v1.0/S2B_MSIL2A_20170924T093019_45_63/S2B_MSIL2A_20170924T093019_45_63.tif,0,2 5 6 10,1
23217,./data/BigEarthNet-v1.0/S2A_MSIL2A_20170816T095031_1_11/S2A_MSIL2A_20170816T095031_1_11.tif,0,18,1
229767,./data/BigEarthNet-v1.0/S2B_MSIL2A_20180326T112109_75_47/S2B_MSIL2A_20180326T112109_75_47.tif,0,2 4 5 6 7 8 13,1
97130,./data/BigEarthNet-v1.0/S2A_MSIL2A_20171104T095201_12_34/S2A_MSIL2A_20171104T095201_12_34.tif,0,0 2 8,1
198670,./data/BigEarthNet-v1.0/S2B_MSIL2A_20170817T101019_44_34/S2B_MSIL2A_20170817T101019_44_34.tif,0,18,1


In [ ]:
class Rsdata(Dataset):
    def __init__(self, images_filepaths, transform=None):
        self.images_filepaths = images_filepaths
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths['fname'].iloc[idx]
        image=open_tif(image_filepath)
        label = self.images_filepaths['labels'].iloc[idx]
        if self.transform is not None:
            result=[]
            for i in range(len(self.transform)):
                result.append(self.transform[i](image=image)['image'])
        return result, label

In [ ]:
dataset = Rsdata(newdf, transform=trainaug)

In [ ]:
data_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=12,
        num_workers=2,
        pin_memory=True,
        drop_last=True,
    )

In [ ]:
xx=first(data_loader)

In [ ]:
def first(x, f=None, negate=False, **kwargs):
    "First element of `x`, optionally filtered by `f`, or None if missing"
    x = iter(x)
    if f: x = filter_ex(x, f=f, negate=negate, gen=True, **kwargs)
    return next(x, None)

In [ ]:
import timm

In [ ]:
model=timm.create_model('xcit_small_12_p8_224',num_classes=0, in_chans=10,drop_path_rate=0.1)

In [ ]:
student = timm.create_model('xcit_small_12_p8_224',num_classes=0, in_chans=10,drop_path_rate=0.1)
teacher = timm.create_model('xcit_small_12_p8_224',num_classes=0, in_chans=10)
embed_dim = student.embed_dim

In [ ]:
from fastai.vision.all import *

In [ ]:
from torchvision import models as torchvision_models


NameError: name 'args' is not defined

In [ ]:
student=xresnet18(c_in=10)
student[11]=nn.Identity()


In [ ]:
student = utils.MultiCropWrapper(student, DINOHead(
    512,
    2**16,
    use_bn=False,
    norm_last_layer=True,
))
# teacher = utils.MultiCropWrapper(
#     teacher,
#     DINOHead(embed_dim, 2**16, use_bn=False),
# )

In [ ]:
temp=torch.ones(1,10,120,120)

In [ ]:
student(temp)

tensor([[ 0.0150,  0.0144, -0.0184,  ..., -0.0575, -0.0004,  0.0712]],
       grad_fn=<MmBackward>)

In [ ]:
def train_dino(args):
    utils.init_distributed_mode(args)
    utils.fix_random_seeds(args.seed)
    print("git:\n  {}\n".format(utils.get_sha()))
    print("\n".join("%s: %s" % (k, str(v)) for k, v in sorted(dict(vars(args)).items())))
    cudnn.benchmark = True

    # ============ preparing data ... ============
    df=pd.read_csv('./data/file.csv')
    newdf=pd.concat([df[df['Isval']==0].sample(frac=0.05,random_state=10),df[df['Isval']==1]])
    newdf['labels']=1
    data_loader = db.dataloaders(source=newdf, bs=args.batch_size_per_gpu, num_workers=args.num_workers,drop_last=True,pin_memory=True)
    data_loader=DistributedDL(data_loader.train)
#     print(f"Data loaded: there are {len(dataset)} images.")

    # ============ building student and teacher networks ... ============
    student = timm.create_model('xcit_small_12_p8_224',num_classes=0, in_chans=10,drop_path_rate=0.1)
    teacher = timm.create_model('xcit_small_12_p8_224',num_classes=0, in_chans=10)
    embed_dim = student.embed_dim
    student = utils.MultiCropWrapper(student, DINOHead(
    embed_dim,
    2**16,
    use_bn=False,
    norm_last_layer=True,))
    teacher = utils.MultiCropWrapper(
    teacher,
    DINOHead(embed_dim, 2**16, use_bn=False))
    # move networks to gpu
    student, teacher = student.cuda(), teacher.cuda()
    # synchronize batch norms (if any)
    if utils.has_batchnorms(student):
        student = nn.SyncBatchNorm.convert_sync_batchnorm(student)
        teacher = nn.SyncBatchNorm.convert_sync_batchnorm(teacher)

        # we need DDP wrapper to have synchro batch norms working...
        teacher = nn.parallel.DistributedDataParallel(teacher, device_ids=[args.gpu])
        teacher_without_ddp = teacher.module
    else:
        # teacher_without_ddp and teacher are the same thing
        teacher_without_ddp = teacher
    student = nn.parallel.DistributedDataParallel(student, device_ids=[args.gpu])
    # teacher and student start with the same weights
    teacher_without_ddp.load_state_dict(student.module.state_dict())
    # there is no backpropagation through the teacher, so no need for gradients
    for p in teacher.parameters():
        p.requires_grad = False
    print(f"Student and Teacher are built: they are both {args.arch} network.")

    # ============ preparing loss ... ============
    dino_loss = DINOLoss(
        args.out_dim,
        args.local_crops_number + 2,  # total number of crops = 2 global crops + local_crops_number
        args.warmup_teacher_temp,
        args.teacher_temp,
        args.warmup_teacher_temp_epochs,
        args.epochs,
    ).cuda()

    # ============ preparing optimizer ... ============
    params_groups = utils.get_params_groups(student)
    if args.optimizer == "adamw":
        optimizer = torch.optim.AdamW(params_groups)  # to use with ViTs
    elif args.optimizer == "sgd":
        optimizer = torch.optim.SGD(params_groups, lr=0, momentum=0.9)  # lr is set by scheduler
    elif args.optimizer == "lars":
        optimizer = utils.LARS(params_groups)  # to use with convnet and large batches
    # for mixed precision training
    fp16_scaler = None
    if args.use_fp16:
        fp16_scaler = torch.cuda.amp.GradScaler()

    # ============ init schedulers ... ============
    lr_schedule = utils.cosine_scheduler(
        args.lr * (args.batch_size_per_gpu * utils.get_world_size()) / 256.,  # linear scaling rule
        args.min_lr,
        args.epochs, len(data_loader),
        warmup_epochs=args.warmup_epochs,
    )
    wd_schedule = utils.cosine_scheduler(
        args.weight_decay,
        args.weight_decay_end,
        args.epochs, len(data_loader),
    )
    # momentum parameter is increased to 1. during training with a cosine schedule
    momentum_schedule = utils.cosine_scheduler(args.momentum_teacher, 1,
                                               args.epochs, len(data_loader))
    print(f"Loss, optimizer and schedulers ready.")

    # ============ optionally resume training ... ============
    to_restore = {"epoch": 0}
    utils.restart_from_checkpoint(
        os.path.join(args.output_dir, "checkpoint.pth"),
        run_variables=to_restore,
        student=student,
        teacher=teacher,
        optimizer=optimizer,
        fp16_scaler=fp16_scaler,
        dino_loss=dino_loss,
    )
    start_epoch = to_restore["epoch"]

    start_time = time.time()
    print("Starting DINO training !")
    for epoch in range(start_epoch, args.epochs):
        data_loader.sampler.set_epoch(epoch)

        # ============ training one epoch of DINO ... ============
        train_stats = train_one_epoch(student, teacher, teacher_without_ddp, dino_loss,
            data_loader, optimizer, lr_schedule, wd_schedule, momentum_schedule,
            epoch, fp16_scaler, args)

        # ============ writing logs ... ============
        save_dict = {
            'student': student.state_dict(),
            'teacher': teacher.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch + 1,
            'args': args,
            'dino_loss': dino_loss.state_dict(),
        }
        if fp16_scaler is not None:
            save_dict['fp16_scaler'] = fp16_scaler.state_dict()
        utils.save_on_master(save_dict, os.path.join(args.output_dir, 'checkpoint.pth'))
        if args.saveckp_freq and epoch % args.saveckp_freq == 0:
            utils.save_on_master(save_dict, os.path.join(args.output_dir, f'checkpoint{epoch:04}.pth'))
        log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                     'epoch': epoch}
        if utils.is_main_process():
            with (Path(args.output_dir) / "log.txt").open("a") as f:
                f.write(json.dumps(log_stats) + "\n")
    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))


In [ ]:
utils.has_batchnorms(student)

True